<a href="https://colab.research.google.com/github/whkaikai/-python-/blob/main/interesting_pytorch_basics_03logisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
# Hyper-parameters 
input_size = 28 * 28    # 784
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [3]:
# MNIST dataset (images and labels)
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../../data/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/MNIST/raw



In [4]:
# Data loader (input pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [5]:
# Logistic regression model
model = nn.Linear(input_size, num_classes)

# Loss and optimizer
# nn.CrossEntropyLoss() computes softmax internally
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

In [6]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Reshape images to (batch_size, input_size)
        images = images.reshape(-1, input_size)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 2.2197
Epoch [1/5], Step [200/600], Loss: 2.1535
Epoch [1/5], Step [300/600], Loss: 2.0330
Epoch [1/5], Step [400/600], Loss: 1.9466
Epoch [1/5], Step [500/600], Loss: 1.9270
Epoch [1/5], Step [600/600], Loss: 1.8110
Epoch [2/5], Step [100/600], Loss: 1.7516
Epoch [2/5], Step [200/600], Loss: 1.7063
Epoch [2/5], Step [300/600], Loss: 1.6696
Epoch [2/5], Step [400/600], Loss: 1.6426
Epoch [2/5], Step [500/600], Loss: 1.5073
Epoch [2/5], Step [600/600], Loss: 1.3501
Epoch [3/5], Step [100/600], Loss: 1.4287
Epoch [3/5], Step [200/600], Loss: 1.3956
Epoch [3/5], Step [300/600], Loss: 1.3962
Epoch [3/5], Step [400/600], Loss: 1.3258
Epoch [3/5], Step [500/600], Loss: 1.2716
Epoch [3/5], Step [600/600], Loss: 1.1873
Epoch [4/5], Step [100/600], Loss: 1.1961
Epoch [4/5], Step [200/600], Loss: 1.1945
Epoch [4/5], Step [300/600], Loss: 1.1558
Epoch [4/5], Step [400/600], Loss: 1.1793
Epoch [4/5], Step [500/600], Loss: 1.2415
Epoch [4/5], Step [600/600], Loss:

In [7]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, input_size)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()

    print('Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the model on the 10000 test images: 82.7300033569336 %


In [8]:
# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')